##AUTOR: DANIEL SOUTO NEVES
##EMAIL: daniel.neves@utp.edu.br
##UNIVERSIDADE TUIUTI DO PARANÁ
##BACHARELADO EM CIÊNCIA DA COMPUTAÇÃO
##DISCIPLINA DE INTELIGÊNCIA COMPUTACIONAL
##TRABALHO 1

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
import numpy as np

KeyboardInterrupt: ignored

In [ ]:
#Funcao fitness -> valor total sem ultrapassar o peso maximo
def fitness(individuo):
 valor = 0
 peso = 0
 i = 0
 while i < len(individuo):
   if individuo[i] == 1:
     valor = valor + itens[i][1]
     peso = peso + itens[i][2]
   i = i + 1
   if peso > pesoMax:
   #Se ultrapassar o peso maximo, sera punido
    return 0
 return valor
 

In [ ]:
#Selecao dos pais

##Aleatoria
def pais_aleatorio(populacao):
  a = np.random.randint(0,len(populacao))
  b = np.random.randint(0,len(populacao))
  while a == b:
    b = np.random.randint(0,len(populacao))
  #Retorna indice dos individuos na populacao
  return [a,b]

##Torneio
def pais_torneio(aptidao):
  k = int(len(aptidao)*0.1)
  #Definido arbitrariamente, mas deve ser <= len(populacao)/2 para que seja possivel
  #realizar os dois torneios, visto que um individuo nao deve participar de ambos

  competidores1 = []
  i = 0
  
  #Escolhe aleatoriamente os indices dos individuos que vao competir
  while i < k:
    indice = np.random.randint(0,len(aptidao))
    while indice in competidores1:
      indice = np.random.randint(0,len(aptidao))
    competidores1.append(indice)
    i = i + 1  
  
  #Converte os indices dos competidores para as aptidoes para determinar o ganhador
  torneio1 = []
  for i in competidores1:
    torneio1.append(aptidao[i])

  #Valor mais proximo de 0 ganha
  vencedor1 = competidores1[torneio1.index(max(torneio1))]

  #Igual acima para o segundo pai, sem repetir aqueles que ja competiram
  competidores2 = []
  i = 0
  while i < k:
    indice = np.random.randint(0,len(aptidao))
    while indice in competidores1 or indice in competidores2:
      indice = np.random.randint(0,len(aptidao))
    competidores2.append(indice)
    i = i + 1  
  
  torneio2 = []
  for i in competidores2:
    torneio2.append(aptidao[i])

  vencedor2 = competidores2[torneio2.index(max(torneio2))]

  #Retorna indice dos individuos na populacao
  return[vencedor1,vencedor2]

##Roleta viciada
#Aptidao define a fatia da roleta para cada individuo
def pais_roleta(aptidao):
  soma = 0
  for i in aptidao:
    soma = soma + i

  peso = []
  for i in aptidao:
    if soma != 0:
      peso.append(100*i/soma)
    else:
      peso.append(100/len(aptidao))

  acumulado = []
  for i in peso:
    if acumulado:
      acumulado.append(acumulado[-1] + i)
    else:
      acumulado.append(i)

  roleta1 = np.random.uniform(0.0,100.0)
  roleta2 = np.random.uniform(0.0,100.0)
  while roleta2 == roleta1:
    roleta2 = np.random.uniform(0.0,100.0)
    
  for i in acumulado:
    if roleta1 < i:
      pai1 = acumulado.index(i)
      break
  
  for i in acumulado:
    if roleta2 < i:
      pai2 = acumulado.index(i) 
      break
  
  return [pai1,pai2]


In [ ]:
#Crossover

##Um ponto de corte
def um_corte(pai1,pai2):

  indice_corte = np.random.randint(1,len(pai1)-1)

  i = 0
  filho1 = []
  filho2 = []

  while i < len(pai1):
    if i <= indice_corte:
      filho1.append(pai1[i])
      filho2.append(pai2[i])
    else:
      filho1.append(pai2[i])
      filho2.append(pai1[i])
    i = i + 1

  filhos = []
  filhos.append(filho1)
  filhos.append(filho2)

  return filhos

##Dois pontos de corte
def dois_corte(pai1,pai2):

  indice_corte1 = np.random.randint(1,len(pai1)-1)
  indice_corte2 = np.random.randint(1,len(pai1)-1)
  while indice_corte2 == indice_corte1:
    indice_corte2 = np.random.randint(1,len(pai1)-1)
  
  segundo_corte = max(indice_corte1,indice_corte2)
  primeiro_corte = min(indice_corte1,indice_corte2)

  i = 0
  filho1 = []
  filho2 = []

  while i < len(pai1):
    if i <= primeiro_corte or i > segundo_corte:
      filho1.append(pai1[i])
      filho2.append(pai2[i])
    else:
      filho1.append(pai2[i])
      filho2.append(pai1[i])
    i = i + 1

  filhos = []
  filhos.append(filho1)
  filhos.append(filho2)

  return filhos

##Uniforme
def uniforme(pai1,pai2):

  i = 0
  filho1 = []
  filho2 = []

  while i < len(pai1):
    if pai1[i] == pai2[i]:
      filho1.append(pai1[i])
      filho2.append(pai2[i])
    else:
      a = np.random.randint(0,2)
      if a == 1:
        filho1.append(pai1[i])
        filho2.append(pai2[i])
      else:
        filho1.append(pai2[i])
        filho2.append(pai1[i])
    i = i + 1

  filhos = []
  filhos.append(filho1)
  filhos.append(filho2)

  return filhos

##Mutacao
def mutacao(individuo):
  i = 0
  while i < len(individuo):
    a = np.random.uniform(0.0,100.0)
    if a <= 1.0:
      if individuo[i] <= muta:
        individuo[i] = 0
      else:
        individuo[i] = 1
    i = i + 1

  return individuo


In [ ]:
#Gerar filhos
def fazer_filhos_pa_um(pop): #Pais aleatorios, crossover de um corte
  filhos = np.random.randint(0,2,(len(pop),len(pop[1])))
  i = 0
  while i < len(pop):
    indice_pais = pais_aleatorio(pop)
    p1 = pop[indice_pais[0]]
    p2 = pop[indice_pais[1]]

    crias = um_corte(p1,p2)
    filhos[i] = mutacao(crias[0])
    filhos[i + 1] = mutacao(crias[1])

    i = i + 2
  
  return filhos
def fazer_filhos_pa_dois(pop): #Pais aleatorios, crossover de dois cortes
  filhos = np.random.randint(0,2,(len(pop),len(pop[1])))
  i = 0
  while i < len(pop):
    indice_pais = pais_aleatorio(pop)
    p1 = pop[indice_pais[0]]
    p2 = pop[indice_pais[1]]

    crias = dois_corte(p1,p2)
    filhos[i] = mutacao(crias[0])
    filhos[i + 1] = mutacao(crias[1])

    i = i + 2
  
  return filhos

def fazer_filhos_pa_uni(pop): #Pais aleatorios, crossover uniforme
  filhos = np.random.randint(0,2,(len(pop),len(pop[1])))
  i = 0
  while i < len(pop):
    indice_pais = pais_aleatorio(pop)
    p1 = pop[indice_pais[0]]
    p2 = pop[indice_pais[1]]

    crias = uniforme(p1,p2)
    filhos[i] = mutacao(crias[0])
    filhos[i + 1] = mutacao(crias[1])

    i = i + 2
  
  return filhos

def fazer_filhos_pt_um(pop,apt): #Pais torneio, crossover de um corte
  filhos = np.random.randint(0,2,(len(pop),len(pop[1])))
  i = 0
  while i < len(pop):
    indice_pais = pais_torneio(apt)
    p1 = pop[indice_pais[0]]
    p2 = pop[indice_pais[1]]

    crias = um_corte(p1,p2)
    filhos[i] = mutacao(crias[0])
    filhos[i + 1] = mutacao(crias[1])

    i = i + 2
  
  return filhos

def fazer_filhos_pt_dois(pop,apt): #Pais torneio, crossover de dois cortes
  filhos = np.random.randint(0,2,(len(pop),len(pop[1])))
  i = 0
  while i < len(pop):
    indice_pais = pais_torneio(apt)
    p1 = pop[indice_pais[0]]
    p2 = pop[indice_pais[1]]

    crias = dois_corte(p1,p2)
    filhos[i] = mutacao(crias[0])
    filhos[i + 1] = mutacao(crias[1])

    i = i + 2
  
  return filhos

def fazer_filhos_pt_uni(pop,apt): #Pais torneio, crossover uniforme
  filhos = np.random.randint(0,2,(len(pop),len(pop[1])))
  i = 0
  while i < len(pop):
    indice_pais = pais_torneio(apt)
    p1 = pop[indice_pais[0]]
    p2 = pop[indice_pais[1]]

    crias = uniforme(p1,p2)
    filhos[i] = mutacao(crias[0])
    filhos[i + 1] = mutacao(crias[1])

    i = i + 2
  
  return filhos

def fazer_filhos_pr_um(pop,apt): #Pais roleta, crossover de um corte
  filhos = np.random.randint(0,2,(len(pop),len(pop[1])))
  i = 0
  while i < len(pop):
    indice_pais = pais_roleta(apt)
    p1 = pop[indice_pais[0]]
    p2 = pop[indice_pais[1]]

    crias = um_corte(p1,p2)
    filhos[i] = mutacao(crias[0])
    filhos[i + 1] = mutacao(crias[1])

    i = i + 2
  
  return filhos

def fazer_filhos_pr_dois(pop,apt): #Pais roleta, crossover de dois cortes
  filhos = np.random.randint(0,2,(len(pop),len(pop[1])))
  i = 0
  while i < len(pop):
    indice_pais = pais_roleta(apt)
    p1 = pop[indice_pais[0]]
    p2 = pop[indice_pais[1]]

    crias = dois_corte(p1,p2)
    filhos[i] = mutacao(crias[0])
    filhos[i + 1] = mutacao(crias[1])

    i = i + 2
  
  return filhos

def fazer_filhos_pr_uni(pop,apt): #Pais roleta, crossover de um cortes
  filhos = np.random.randint(0,2,(len(pop),len(pop[1])))
  i = 0
  while i < len(pop):
    indice_pais = pais_roleta(apt)
    p1 = pop[indice_pais[0]]
    p2 = pop[indice_pais[1]]

    crias = uniforme(p1,p2)
    filhos[i] = mutacao(crias[0])
    filhos[i + 1] = mutacao(crias[1])

    i = i + 2
  
  return filhos

In [ ]:
#Selecao dos sobreviventes
##Aleatoria
def sobrev_aleator(pop,fil):
  aleatorio_pais = []
  aleatorio_filhos = []
  i = 0
  while i < len(pop)/2:
    p = np.random.randint(0,len(pop))
    f = np.random.randint(0,len(fil))
    if not aleatorio_pais:
      aleatorio_pais.append(p)
      aleatorio_filhos.append(f)
    else:
      while p in aleatorio_pais:
        p = np.random.randint(0,len(pop))
      while f in aleatorio_filhos:
        f = np.random.randint(0,len(fil))
      aleatorio_pais.append(p)
      aleatorio_filhos.append(f)
    i = i + 1
  
  sobrev = np.random.randint(0,2,(len(pop),len(pop[1])))
  i = 0
  j = 0
  while i < len(sobrev):
    sobrev[i] = pop[aleatorio_pais[j]]
    sobrev[i + 1] = fil[aleatorio_filhos[j]]
    j = j + 1
    i = i + 2
  
  return sobrev

##Ranking
from operator import itemgetter
def sobrev_rank(pop,apt_pop,fil,apt_fil):
  i = 0
  popi = []
  while i < len(pop):
    popi.append([apt_pop[i],pop[i]])
    i = i + 1
  
  ord_pop = sorted(popi, key=itemgetter(0),reverse=True)
  
  i = 0
  fili = []
  while i < len(fil):
    fili.append([apt_fil[i],fil[i]])
    i = i + 1
  
  ord_fil = sorted(fili, key=itemgetter(0),reverse=True)

  todos = ord_pop + ord_fil
  sob = sorted(todos, key=itemgetter(0),reverse=True)

  sobrev = np.random.randint(0,2,(len(pop),len(pop[1])))
    
  i = 0
  usados = []
  while i < len(sob)/2:
    if i < len(sob)/8:
      sobrev[i] = sob[i][1]
    else:
      ind = np.random.randint(len(sob)/4,len(sob))
      while ind in usados:
        ind = np.random.randint(len(sob)/4,len(sob))
      usados.append(ind)
      sobrev[i] = sob[ind][1]  
    i = i + 1

  return sobrev    

##Elitismo
def sobrev_elite(pop,apt_pop,fil,apt_fil):
  i = 0
  popi = []
  while i < len(pop):
    popi.append([apt_pop[i],pop[i]])
    i = i + 1
  
  ord_pop = sorted(popi, key=itemgetter(0),reverse=True)
  
  i = 0
  fili = []
  while i < len(fil):
    fili.append([apt_fil[i],fil[i]])
    i = i + 1
  
  ord_fil = sorted(fili, key=itemgetter(0),reverse=True)

  todos = ord_pop + ord_fil
  ord_todos = sorted(todos, key=itemgetter(0),reverse=True)

  sob = ord_todos[:len(ord_todos)-len(ord_pop)]
  
  sobrev = np.random.randint(0,2,(len(pop),len(pop[1])))
    
  i = 0
  usados = []
  while i < len(sob):
    sobrev[i] = sob[i][1]
    i = i + 1

  return sobrev

In [ ]:
print('Digite o caminho do arquivo de dados:')
filename = input()
#/content/gdrive/My Drive/Colab Notebooks/IC/ks_i0012.txt
#/content/gdrive/My Drive/Colab Notebooks/IC/ks_i0032.txt
#/content/gdrive/My Drive/Colab Notebooks/IC/ks_i0100.txt
#/content/gdrive/My Drive/Colab Notebooks/IC/ks_i0322.txt
#/content/gdrive/My Drive/Colab Notebooks/IC/ks_i0450.txt
#/content/gdrive/My Drive/Colab Notebooks/IC/ks_i0890.txt
#/content/gdrive/My Drive/Colab Notebooks/IC/ks_i1200.txt
#/content/gdrive/My Drive/Colab Notebooks/IC/ks_i8000.txt

#Le arquivo com todos os itens e o peso maximo da mochila
with open(filename,'r') as arq:
 pesoMax = int(arq.readline())
 L = arq.readlines()

itensSTR = []
itens = []
i = 0

#Cria uma lista onde cada posicao possui o codigo, valor e peso um item
for a in L:
 itensSTR.append(a.split()) 
 itens.append([int(itensSTR[i][0]),float(itensSTR[i][1]),float(itensSTR[i][2])])
 i = i + 1

num_itens = len(itens)


print("Digite o tamanho da populacao inicial:")
pop_size = int(input())

#Define condicao de parada
print("Digite o numero de geracoes desejado:")
geracao_max= int(input())

#Fenotipo: Cada gene diz se o item da posicao correspondente ao seu
#locus no frame se encontra na mochila (1) ou nao (0)

print("Selecione o metodo de selecao dos pais:")
print("1. Aleatorio")
print("2. Torneio")
print("3. Roleta Viciada")
sele_pais = int(input())

print("Selecione o metodo de crossover:")
print("1. Um ponto de corte")
print("2. Dois pontos de corte")
print("3. Uniforme")
sele_cross = int(input())

print("Digite a taxa de mutacao: ")
muta = float

print("Selecione o metodo de selecao dos Sobreviventes:")
print("1. Aleatorio")
print("2. Ranking")
print("3. Elitismo")
sele_sobr = int(input())

Digite o caminho do arquivo de dados:
/content/gdrive/My Drive/Colab Notebooks/IC/ks_i0450.txt
Digite o tamanho da populacao inicial:
2500
Digite o numero de geracoes desejado:
50
Selecione o metodo de selecao dos pais:
1. Aleatorio
2. Torneio
3. Roleta Viciada
2
Selecione o metodo de crossover:
1. Um ponto de corte
2. Dois pontos de corte
3. Uniforme
1
Selecione o metodo de selecao dos Sobreviventes:
1. Aleatorio
2. Ranking
3. Elitismo
2


In [ ]:
#Gera a populacao inicial
geracao = 0

if num_itens <= 100:
  pop = np.random.randint(0,2,(pop_size,num_itens))
else: 
  #Chance de estourar o peso máximo é muito mais alta para uma variedade maior de itens
  import random
  if num_itens < 1000:
     itens_inicio = int(num_itens/(num_itens*0.1))
  else:
    itens_inicio = int(num_itens/(num_itens*0.01))
  pop = np.random.randint(0,2,(pop_size,num_itens))
  i = 0
  while i < pop_size:
    indi = np.array(([0]*(num_itens-itens_inicio) + [1]*itens_inicio))
    if len(indi) < num_itens: #Caso a divisao por inteiro corte um ou mais items
      while len(indi) < len(indi):
        indi.append(random.randint(0,2))
    random.shuffle(indi)
    pop[i] = indi

    i = i + 1

f = open("Evolucao_mochila.txt","w")
f.write("Evolucao:\n")
f.close()

while geracao < geracao_max:

  print("Comecando geracao ",geracao,"...")
  #Avaliar a aptidao da populacao
  aptidao = []
  for i in pop:
    aptidao.append(fitness(i))

  iteracao = list(zip(aptidao,pop))
  print("Melhor individuo: ")
  print(sorted(iteracao,key=itemgetter(0),reverse=True)[0])  

  f = open("Evolucao_mochila.txt","a")
  f.write("\n")
  f.write(str(geracao))
  f.write("\n")
  f.write("\n".join("%s %s" % x for x in iteracao))
  f.write("\n")
  f.close()

  #Gerar filhos
  if sele_pais == 1 and sele_cross == 1:
    filhos = fazer_filhos_pa_um(pop)

  if sele_pais == 1 and sele_cross == 2:
    filhos = fazer_filhos_pa_dois(pop)

  if sele_pais == 1 and sele_cross == 3:
    filhos = fazer_filhos_pa_uni(pop)

  if sele_pais == 2 and sele_cross == 1:
    filhos = fazer_filhos_pt_um(pop,aptidao)

  if sele_pais == 2 and sele_cross == 2:
    filhos = fazer_filhos_pt_dois(pop,aptidao)

  if sele_pais == 2 and sele_cross == 3:
    filhos = fazer_filhos_pt_uni(pop,aptidao)

  if sele_pais == 3 and sele_cross == 1:
    filhos = fazer_filhos_pr_um(pop,aptidao)

  if sele_pais == 3 and sele_cross == 2:
    filhos = fazer_filhos_pr_dois(pop,aptidao)

  if sele_pais == 3 and sele_cross == 3:
    filhos = fazer_filhos_pr_uni(pop,aptidao)

  #Avaliar aptidao dos filhos
  aptidao_filhos = []
  for i in filhos:
    aptidao_filhos.append(fitness(i))

  #Selecionar proxima populacao
  if sele_sobr == 1:
    pop = sobrev_aleator(pop,filhos)
  if sele_sobr == 1:
    pop = sobrev_rank(pop,aptidao,filhos,aptidao_filhos)
  if sele_sobr == 1:  
    pop = sobrev_elite(pop,aptidao,filhos,aptidao_filhos)
  
  print("\nGeracao", geracao," completada!")
  print("\n")

  geracao = geracao + 1


aptidao = []
for i in pop:
  aptidao.append(fitness(i))

iteracao = list(zip(aptidao,pop))
print("Melhor individuo: ")
print(sorted(iteracao,key=itemgetter(0),reverse=True)[0])  
 
f = open("Evolucao_mochila.txt","a")
f.write("\n")
f.write(str(geracao))
f.write("\n")
f.write("\n".join("%s %s" % x for x in iteracao))
f.write("\n")
f.close()

print("Programa Terminado!")
#f = open("Evolucao_mochila.txt","r")
#print(f.read())
#f.close()

Comecando geracao  0 ...
Melhor individuo: 
(3976.6000000000004, array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0]))

Geracao 0  completada!


Comecando geracao  1 ...
Melhor individuo: 
(3976.6000000000004, array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0]))

Geracao 1  completada!


Comecando 

KeyboardInterrupt: ignored